### Setup your environnement variables

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
os.environ["UIFORM_API_KEY"] = "YOUR API KEY" # go to https://uiform.com to create your API Key

# Get started

In [2]:
from dotenv import load_dotenv
assert load_dotenv() # Load environment variables from a .env file

In [3]:
from uiform import UiForm
uiclient = UiForm()
uiclient.models.list()

['claude-3-5-sonnet-latest',
 'claude-3-5-sonnet-20241022',
 'claude-3-opus-20240229',
 'claude-3-sonnet-20240229',
 'claude-3-haiku-20240307',
 'gpt-4o',
 'gpt-4o-mini',
 'chatgpt-4o-latest',
 'gpt-4o-2024-11-20',
 'gpt-4o-2024-08-06',
 'gpt-4o-2024-05-13',
 'gpt-4o-mini-2024-07-18',
 'o1-2024-12-17',
 'o1-mini-2024-09-12',
 'grok-2-vision-1212',
 'grok-2-1212',
 'gemini-2.0-flash-exp',
 'gemini-1.5-flash-8b',
 'gemini-1.5-flash',
 'gemini-1.5-pro']

In [4]:
from uiform import UiForm
from openai import OpenAI

uiclient = UiForm(api_key = "sk_3ovI3g3Hq7pmBvySp3tP2OCMlZtHSNsAXfKG6VQjiDI")

doc_msg = uiclient.documents.create_messages(document = "freight/booking_confirmation.jpg")


In [5]:
schema_obj.schema_version

NameError: name 'schema_obj' is not defined

In [10]:
import os
from uiform import UiForm
from openai import OpenAI


doc_msg = uiclient.documents.create_messages(document = "freight/booking_confirmation.jpg")

client = OpenAI(base_url="https://api.x.ai/v1", api_key=os.getenv("XAI_API_KEY"))
completion = client.chat.completions.create(
    model="grok-2-vision-1212",
    messages=doc_msg.openai_messages + [
        {
            "role": "user",
            "content": "Summarize the document"
        }
    ]
)

print(completion.model_dump())


{'id': 'd42502fd-24d5-4287-9636-a5e4b6c7c85e', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '**Document Summary:**\n\n**Title:** Confirmation d\'affrètement (Charter Confirmation)\n\n**Company:** ACME Corporation\n\n**Details:**\n- **Date:** 11/8/2024, 11:41:43 AM\n- **VAT Number:** GB123456789\n\n**Company Contact:**\n- **Name:** Jane Doe\n- **Phone:** +44 20 7946 0958\n- **Email:** client@acme.com\n\n**Site Details:**\n- **Address:** 456 Oak Avenue, Floor 3, Manchester, M1 4WP, United Kingdom\n\n**Billing Address:**\n- **Address:** 123 Elm Street, Suite 500, London, WC2N 5DU GB\n- **Phone:** +44 20 7946 0958\n- **Email:** client@acme.com\n- **VAT Number:** GB123456789\n\n**Carrier:**\n- **Name:** TRANSPORTEXPRESS\n- **Service:** Service Logistique\n- **Address:** 45 rue Industrielle Est, Bâtiment C, Les Landes Transporteurs, 92130 PARIS, France\n- **Phone:** +33 1 23 45 67 90\n- **Fax:** +33 1 23 45 67 90\n\n**Carrier Commitments:**\n- Su

In [6]:



# ------------------------------------------------------------------------------------------------
# Structured extraction with OpenAI API
client = OpenAI(base_url="https://api.x.ai/v1", api_key=os.getenv("XAI_API_KEY"))


from uiform import Schema

schema_obj = Schema(
    json_schema = {
        'X-SystemPrompt': 'You are a useful assistant extracting information from documents.',
        'properties': {
            'name': {
                'X-FieldPrompt': 'Provide a descriptive and concise name for the event.',
                'description': 'The name of the calendar event.',
                'title': 'Name',
                'type': 'string'
            },
            'date': {
                'X-ReasoningPrompt': 'The user can mention it in any format, like **next week** or **tomorrow**. Infer the right date format from the user input.',
                'description': 'The date of the calendar event in ISO 8601 format.',
                'title': 'Date',
                'type': 'string'
            }
        },
        'required': ['name', 'date'],
        'title': 'CalendarEvent',
        'type': 'object'
    }
)

completion = client.chat.completions.create(
    model="grok-2-vision-1212",
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": schema_obj.schema_version,
            "schema": schema_obj.inference_json_schema,
            "strict": True
        }
    }
)
print(completion)

ChatCompletion(id='196c9b21-b518-402b-ac98-5543e18d54ad', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "name": "Transport of Dangerous Goods",\n  "reasoning___date": "From 5/2/2023 to 5/3/2023",\n  "date": "2023-02-05T12:00:00Z/2023-03-05T16:00:00Z"\n}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736929087, model='grok-2-vision-1212', object='chat.completion', service_tier=None, system_fingerprint='fp_958d152a61', usage=CompletionUsage(completion_tokens=89, prompt_tokens=1895, total_tokens=1984, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0, text_tokens=103, image_tokens=1792)))


In [7]:

# --- OR 

completion = client.beta.chat.completions.parse(
    model="grok-2-vision-1212",
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format=schema_obj.inference_pydantic_model
)


In [10]:
import os

doc_msg = uiclient.documents.create_messages(document = "freight/booking_confirmation.jpg", modality="text")


client = OpenAI(
    api_key = os.getenv("GEMINI_API_KEY"),
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
)
completion = client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": schema_obj.schema_version,
            "schema": schema_obj.inference_gemini_json_schema, # Very important ! 
            # We need to put the inference_gemini_json_schema here because gemini does not allow nullable fields,
            # every field is required and the anyOf is not supported.
            "strict": True
        }
    }
)

In [11]:
completion

ChatCompletion(id=None, choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"reasoning___date": "5/2/2023", "date": "2023-05-02", "name": "Goods Transportation"}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736929190, model='gemini-1.5-flash', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=38, prompt_tokens=1416, total_tokens=1454, completion_tokens_details=None, prompt_tokens_details=None))

# Example 1 - Analyze informations from a document

#### Option A - You use our preprocess endpoint and openAI client

In [17]:
import json
from uiform import UiForm, Schema
from openai import OpenAI

with open("freight/schema.json", "r") as f:
    json_schema = json.load(f)


uiclient = UiForm()
doc_msg = uiclient.documents.create_messages(
    document = "freight/booking_confirmation.jpg",
)

schema_obj = Schema(
    json_schema = json_schema
)


# Now you can use your favorite model to analyze your document
client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-4o-mini", 
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format=schema_obj.response_format_json
)

# Validate the response against the original schema if you want to remove the reasoning fields
assert completion.choices[0].message.content is not None
extraction = schema_obj.pydantic_model.model_validate_json(
    completion.choices[0].message.content
)

print(extraction)

booking_id='SHIP-001' payment=TransportPriceData(total_price=1500.0, currency='EUR') client=ClientData(company_name='ACME Corporation', VAT_number=None, city='London', postal_code='WC2N 5DU', country='GB', code='BC-67890', email='client@acme.com') shipments=[ShipmentData(shipment_id='SHIP-001', sender=SenderData(company_name='ACME Corporation', address=AddressDataSimple(city='London', postal_code='WC2N 5DU', country='GB', line1='123 Elm Street', line2='Suite 500'), phone_number='+442079460958', email_address='client@acme.com', pickup_datetime=PickupDatetimeData(date='2023-02-05', start_time='08:00:00', end_time='12:00:00'), observations='Reference: SHIP-001'), recipient=RecipientData(company_name='Beta Industries', address=AddressDataSimple(city='Munich', postal_code='80331', country='DE', line1='789 Pine Street', line2=None), phone_number='+4989123456', email_address=None, delivery_datetime=DeliveryDatetimeData(date='2023-03-05', start_time='10:00:00', end_time='16:00:00'), observatio

#### You can also add some preprocessing operations to find informations that the LLM struggles to parse

In [20]:
text_operations = {
    'regex_instructions' : [
        {
            "name": "vat_number",
            "pattern": r"[Ff][Rr]\s*(\d\s*){11}",
            "description": "VAT number in the format XX999999999"
        }
    ]
}

doc_msg = uiclient.documents.create_messages(
    document = "freight/booking_confirmation.jpg",
    text_operations=text_operations
)

#### Option B - Using UiForm `extract` endpoint

In [21]:
import json
from uiform.client import UiForm

uiclient = UiForm()
completion = uiclient.documents.extractions.parse(
    json_schema = "freight/schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini",
    temperature=0
)

print(completion.choices[0].message.parsed)

booking_id='SHIP-001' payment=TransportPriceData(total_price=1500.0, currency='EUR') client=ClientData(company_name='ACME Corporation', VAT_number=None, city='London', postal_code='WC2N 5DU', country='GB', code=None, email='client@acme.com') shipments=[ShipmentData(shipment_id='SHIP-001', sender=SenderData(company_name='ACME Corporation', address=AddressDataSimple(city='London', postal_code='WC2N 5DU', country='GB', line1='123 Elm Street', line2='Suite 500'), phone_number='+442079460958', email_address='client@acme.com', pickup_datetime=PickupDatetimeData(date='2023-02-05', start_time='08:00:00', end_time='12:00:00'), observations=None), recipient=RecipientData(company_name='Beta Industries', address=AddressDataSimple(city='Munich', postal_code='80331', country='DE', line1='789 Pine Street', line2=None), phone_number='+4989123456', email_address=None, delivery_datetime=DeliveryDatetimeData(date='2023-03-05', start_time='10:00:00', end_time='16:00:00'), observations=None), goods=GoodsDa

## Appendix A - Use text as modality

In [22]:
from uiform.client import UiForm

uiclient = UiForm()

completion = uiclient.documents.extractions.parse(
    json_schema = "freight/schema.json",
    document="freight/booking_confirmation.jpg",
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    modality='text'
)

print(completion.choices[0].message.parsed)

booking_id='SHIP-001' payment=TransportPriceData(total_price=1500.0, currency='EUR') client=ClientData(company_name='ACME Corporation', VAT_number=None, city='London', postal_code='WC2N S5DU', country='GB', code=None, email='client@acme.com') shipments=[ShipmentData(shipment_id='SHIP-001', sender=SenderData(company_name='ACME Corporation', address=AddressDataSimple(city='Manchester', postal_code='M1 4WP', country='GB', line1='456 Oak Avenue', line2='Floor 3 Suite 500'), phone_number='+442079460958', email_address='client@acme.com', pickup_datetime=PickupDatetimeData(date='2023-02-05', start_time='08:00:00', end_time='12:00:00'), observations=None), recipient=RecipientData(company_name='Beta Industries', address=AddressDataSimple(city='Munich', postal_code='80331', country='DE', line1='789 Pine Street', line2=None), phone_number='+4989123456', email_address=None, delivery_datetime=DeliveryDatetimeData(date='2023-03-05', start_time='10:00:00', end_time='16:00:00'), observations=None), go

## Appendix B - Add text operations

In [ ]:
from uiform.client import UiForm

uiclient = UiForm()

text_operations = {
    'regex_instructions' : [
        {
            "name": "vat_number",
            "pattern": r"[Ff][Rr]\s*(\d\s*){11}",
            "description": "VAT number in the format XX999999999"
        }
    ]
}

completion = uiclient.documents.extractions.parse(
    json_schema = "freight/schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    text_operations=text_operations
)

print(completion.choices[0].message.parsed)